<a href="https://colab.research.google.com/github/sehamhemdan/Text-Classification/blob/main/Research2_twitterDataset_Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  del sys.path[0]


In [ ]:
tf.config.list_physical_devices('GPU') 

[]

In [ ]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
working_dir_path ='/content/drive/MyDrive/stanford_sentiment_140.csv'
df = pd.read_csv(working_dir_path , encoding =DATASET_ENCODING , names=DATASET_COLUMNS)


Mounted at /content/drive


In [ ]:
df.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.shape

(1600000, 6)

In [ ]:
df.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [ ]:
df=df.drop(['ids', 'date', 'flag', 'user'],axis=1)

In [ ]:
df

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df.shape

(14640, 2)

In [ ]:
df.columns

Index(['target', 'text'], dtype='object')

In [ ]:
# Rename the columns of df2_melted: df2_tidy
#df2_tidy = df2_melted.rename(columns = {'variable': 'Year', 'value': 'Income'}, inplace = False)
##df=df.rename(columns={'text':'Original_tweet'},inplace=False)
#df.head()

In [ ]:
# write function for removing @user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [ ]:
# create new column with removed @user
df['new_text'] = np.vectorize(remove_pattern)(df['text'], '@[\w]*')

In [ ]:
df.head(10)

,target,text,new_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","http://twitpic.com/2y1zl - Awww, that's a bum..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to s...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am..."
5,0,@Kwesidei not the whole crew,not the whole crew
6,0,Need a hug,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,"hey long time no see! Yes.. Rains a bit ,onl..."
8,0,@Tatiana_K nope they didn't have it,nope they didn't have it
9,0,@twittera que me muera ?,que me muera ?


In [ ]:
import re
df['new_text'] = df['new_text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [ ]:
df.head(3)

,target,text,new_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","http://twitpic.com/2y1zl - Awww, that's a bum..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to s...


In [ ]:
# remove special characters, numbers, punctuations


df['new_text'] = df['new_text'].str.replace('[^a-zA-Z#]+',' ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [ ]:
df.head(10)

,target,text,new_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",http twitpic com y zl Awww that s a bummer Yo...
1,0,is upset that he can't update his Facebook by ...,is upset that he can t update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball Managed to sa...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why am i ...
5,0,@Kwesidei not the whole crew,not the whole crew
6,0,Need a hug,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time no see Yes Rains a bit only a b...
8,0,@Tatiana_K nope they didn't have it,nope they didn t have it
9,0,@twittera que me muera ?,que me muera


In [ ]:
# remove short words
df['new_text'] = df['new_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

In [ ]:
df.head(2)

,target,text,new_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",http twitpic com Awww that bummer You shoulda ...
1,0,is upset that he can't update his Facebook by ...,upset that can update his Facebook texting and...


In [ ]:
#remove stopwords

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')+["because","early","told","teld","everybody","include"]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['new_text'].apply(lambda x: [item for item in x if item not in stop])
df.head(2)

,target,text,new_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",http twitpic com Awww that bummer You shoulda ...
1,0,is upset that he can't update his Facebook by ...,upset that can update his Facebook texting and...


In [ ]:
# create new variable tokenized tweet 
tokenized_tweet = df['new_text'].apply(lambda x: x.split())

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

# apply stemmer for tokenized_tweet
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
# join tokens into one sentence
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
# change df['Tweet'] to tokenized_tweet

In [ ]:
df['new_text']  = tokenized_tweet


In [ ]:
df.head(10)

,target,text,new_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",http twitpic com awww that bummer you shoulda ...
1,0,is upset that he can't update his Facebook by ...,upset that can updat hi facebook text and migh...
2,0,@Kenichan I dived many times for the ball. Man...,dive mani time for the ball manag save the res...
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi and like it fire
4,0,"@nationwideclass no, it's not behaving at all....",not behav all mad whi here becaus can see you ...
5,0,@Kwesidei not the whole crew,not the whole crew
6,0,Need a hug,need hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time see ye rain bit onli bit lol fin...
8,0,@Tatiana_K nope they didn't have it,nope they didn have
9,0,@twittera que me muera ?,que muera


In [ ]:
df.isnull().sum()

target      0
text        0
new_text    0
dtype: int64

In [ ]:
df.columns

Index(['target', 'text', 'new_text'], dtype='object')

In [ ]:
df=df.drop(['text'],axis=1)

In [ ]:
df

,target,new_text
0,0,http twitpic com awww that bummer you shoulda ...
1,0,upset that can updat hi facebook text and migh...
2,0,dive mani time for the ball manag save the res...
3,0,whole bodi feel itchi and like it fire
4,0,not behav all mad whi here becaus can see you ...
...,...,...
1599995,4,just woke have school the best feel ever
1599996,4,thewdb com veri cool hear old walt interview h...
1599997,4,are you readi for your mojo makeov ask for detail
1599998,4,happi birthday boo alll time tupac amaru shakur


# **`code to save preprocessed data to local`**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('preprocessedStanford_sentiment.csv', index = False)
!cp preprocessedStanford_sentiment.csv "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from google.colab import files
df.to_csv('preprocessedStanford_sentiment.csv', index = False)
files.download('preprocessedStanford_sentiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>